In [1]:
import numpy as np
import pandas  as pd

In [2]:
df = pd.read_csv('metadata_clean.csv')
df.head()

,title,genres,runtime,vote_average,vote_count,year
0,Toy Story,"['animation', 'comedy', 'family']",81.0,7.7,5415.0,1995
1,Jumanji,"['adventure', 'fantasy', 'family']",104.0,6.9,2413.0,1995
2,Grumpier Old Men,"['romance', 'comedy']",101.0,6.5,92.0,1995
3,Waiting to Exhale,"['comedy', 'drama', 'romance']",127.0,6.1,34.0,1995
4,Father of the Bride Part II,['comedy'],106.0,5.7,173.0,1995


In [3]:
orig_df = pd.read_csv('movies_metadata.csv')
df['overview'], df['id'] = orig_df['overview'], orig_df['id']
df.head()

D:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,title,genres,runtime,vote_average,vote_count,year,overview,id
0,Toy Story,"['animation', 'comedy', 'family']",81.0,7.7,5415.0,1995,"Led by Woody, Andy's toys live happily in his ...",862
1,Jumanji,"['adventure', 'fantasy', 'family']",104.0,6.9,2413.0,1995,When siblings Judy and Peter discover an encha...,8844
2,Grumpier Old Men,"['romance', 'comedy']",101.0,6.5,92.0,1995,A family wedding reignites the ancient feud be...,15602
3,Waiting to Exhale,"['comedy', 'drama', 'romance']",127.0,6.1,34.0,1995,"Cheated on, mistreated and stepped on, the wom...",31357
4,Father of the Bride Part II,['comedy'],106.0,5.7,173.0,1995,Just when George Banks has recovered from his ...,11862


### Plot Description Based Recommender
- Obtain the data required to build the model.
- Create TF-IDF(Term Frequency-Inverse Document Frequency) vectors for the plot description(df['overview']) of every movie.
- Compute Cosine Similarity score of every movie.
- Write recommendor function that takes in a movie title as an argument and outputs movies most similar to it based on the plot.

### Creating TF-IDF matrix

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words = 'english')

# Replaceing Nan with empty strings
df['overview'] = df['overview'].fillna('')

tfidf_matrix = tfidf.fit_transform(df['overview'])
tfidf_matrix.shape

(45466, 75827)

###  Computing Cosine Similarity

In [5]:
from sklearn.metrics.pairwise import linear_kernel
cos_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

### Creating the Recommender Function

In [6]:
indices = pd.Series(df.index, index = df['title']).drop_duplicates()

Steps involved in building the recommendor function:
- Declare title of the movie as a n argument
- Obtain index of movie from indices reverse mapping
- Get list of cosine similarity scores for thaqt particular movie
- sort the list of tuples on the basis of cosine similarity scores
- Get top 10 elements of the list
- Return the titles corresponding to the indices of the top 10 elements, excluding first

In [7]:
def content_recommendor(title, cosine_sim = cos_sim, df = df, indices = indices):
    idx = indices[title]
    
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    sim_scores = sorted(sim_scores, key = lambda x : x[1], reverse = True)
    
    sim_scores = sim_scores[1:11]
    
    movie_indices = [i[0] for i in sim_scores]
    
    return df['title'].iloc[movie_indices]

In [8]:
content_recommendor('The Lion King')

34682    How the Lion Cub and the Turtle Sang a Song
9353                                The Lion King 1½
9115                  The Lion King 2: Simba's Pride
42829                                           Prey
25654                                 Fearless Fagan
17041                                   African Cats
27933              Massaï, les guerriers de la pluie
6094                                       Born Free
37409                                     Sour Grape
3203                                The Waiting Game
Name: title, dtype: object

In [11]:
content_recommendor('Toy Story')

15348                                     Toy Story 3
2997                                      Toy Story 2
10301                          The 40 Year Old Virgin
24523                                       Small Fry
23843                     Andy Hardy's Blonde Trouble
29202                                      Hot Splash
43427                Andy Kaufman Plays Carnegie Hall
38476    Superstar: The Life and Times of Andy Warhol
42721    Andy Peters: Exclamation Mark Question Point
8327                                        The Champ
Name: title, dtype: object

In [12]:
content_recommendor('Inception')

44314                                 III
2039                                House
25299                       Borrowed Time
2114                    The Farmer's Wife
44792                            Altitude
22619                    The Monkey's Paw
3424     What Ever Happened to Baby Jane?
349                                  Cobb
37187                      Straight Story
8988                           Stone Cold
Name: title, dtype: object